In [1]:
import torch

In [ ]:
# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


def get_padding2d(input_images):
    #padded_images = # добавить нулей с четырех сторон каждого изображения
    ## Top button
    t=list(input_images.size())
    t[2]=1
    newrow=torch.zeros(t)
    newrow=newrow.type(torch.long)
    input_images=input_images.type(torch.long)
    
    input_images=torch.cat((newrow, input_images), 2)
    input_images=torch.cat((input_images,newrow), 2)
    input_images


    ## Sides
    t=list(input_images.size())
    t[-1]=1
    newrow=torch.zeros(t)
    newrow=newrow.type(torch.long)
    input_images=input_images.type(torch.long)
    
    input_images=torch.cat((newrow, input_images), 3)
    input_images=torch.cat((input_images, newrow), 3)
    padded_images=input_images
    
    padded_images=padded_images.type(torch.float)
    return padded_images

In [ ]:
get_padding2d(input_images)

## My Conv2d

In [2]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


In [ ]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

kernel_w=kernel.shape[-1]
kernel_h=kernel.shape[-2]
print(kernel_w,kernel_h)

In [12]:
# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):

    def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
        batch_size, channels_count, input_height, input_width = input_matrix_shape
        output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
        output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    
        return batch_size, out_channels, output_height, output_width

    def out_image_shape(self, input_height, input_width, kernel_size, stride, padding):
        output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
        output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1
        return output_height, output_width

    def create_empty_out_img(self, input_tensor):
        self.input_width = input_tensor.shape[-1]
        self.input_height = input_tensor.shape[-2]
        # in_chanels = input_tensor.shape[-3]
        
        padding = 0
        self.output_height, self.output_width = self.out_image_shape(self.input_height, self.input_width, self.kernel_size, self.stride, padding)
        
        out_tesor = torch.zeros([self.out_channels, self.output_height, self.output_width])
        return out_tesor

    def conv_one_sample(self, input_tensor):
        # W
        self.kernel_w= self.kernel_size #self.kernel.shape[-1]
        # w_otstup = self.kernel_w//2
        # w_ending = self.input_height - w_otstup +1
        # print(f"w_otstup {w_otstup}")

        # H
        self.kernel_h= self.kernel_size # self.kernel.shape[-2]
        # h_otstup = self.kernel_h//2
        # h_ending = self.input_width - h_otstup +1
        # print(f"h_otstup {h_otstup}")

        print("\n\n")
        
        h_start = 0
        for _ in range(self.output_height): #h_centr
            w_start = 0
            for _ in range(self.output_width): # w_centr
                print(f"self.kernel_size {self.kernel_size}")
                window_sliced = input_tensor[:,   h_start:h_start+self.kernel_size,   w_start: w_start+self.kernel_size]
                print(f"window_sliced {window_sliced}")
                w_start+=self.stride
                print(f"\n\n")
            h_start+=self.stride
            
        
        
    def __call__(self, input_tensor):
        # output_tensor = # Напишите в этом месте вычисление свертки с использованием циклов.

        # create
        self.out_tensor=self.create_empty_out_img(input_tensor)
        self.conv_one_sample(input_tensor)

        # return out_tesor

        
        # # проверка является ли input батчем ( dimension = 4 => batch, dimension = 3 => one_image)
        # if len(input_tensor.shape) = 4:
        #     for i in input_tensor.shape[0]:
        # return output_tensor


In [ ]:
# kernel_size = 5
# otstup = kernel_size//2
# stride = 2
# otstup

In [ ]:
# length=9

# begin = otstup
# ending = length - otstup +1
# print(f"begin {begin}")
# print(f"ending {ending}")


# for i in range(otstup , ending, stride):
#     print(i)

In [15]:
img = torch.randint(10, ( 3, 4, 5)).to(torch.float32)
img

tensor([[[9., 8., 7., 7., 6.],
         [1., 3., 7., 1., 1.],
         [3., 8., 7., 3., 3.],
         [4., 4., 5., 7., 0.]],

        [[7., 7., 5., 7., 4.],
         [3., 4., 2., 4., 3.],
         [7., 4., 1., 6., 6.],
         [7., 0., 3., 7., 7.]],

        [[8., 6., 3., 3., 5.],
         [3., 7., 5., 3., 9.],
         [1., 0., 1., 3., 6.],
         [0., 2., 4., 4., 8.]]])

In [ ]:
img[:3,:3]

In [ ]:
img[:,0:2,3:4]

## Get empty tensor

In [16]:
in_channels = 3
out_channels = 4
kernel_size = 2
stride = 1


my_conv = Conv2dLoop(in_channels, out_channels, kernel_size, stride)
# my_conv.set_kernel(kernel)

# my_conv.create_empty_out_img(img)

# img = torch.randint(10, ( 3, 7, 10)).to(torch.float32)
# my_conv(img)

In [ ]:
img

## Call my conv3d

In [17]:
my_conv(img)




self.kernel_size 2
window_sliced tensor([[[9., 8.],
         [1., 3.]],

        [[7., 7.],
         [3., 4.]],

        [[8., 6.],
         [3., 7.]]])



self.kernel_size 2
window_sliced tensor([[[8., 7.],
         [3., 7.]],

        [[7., 5.],
         [4., 2.]],

        [[6., 3.],
         [7., 5.]]])



self.kernel_size 2
window_sliced tensor([[[7., 7.],
         [7., 1.]],

        [[5., 7.],
         [2., 4.]],

        [[3., 3.],
         [5., 3.]]])



self.kernel_size 2
window_sliced tensor([[[7., 6.],
         [1., 1.]],

        [[7., 4.],
         [4., 3.]],

        [[3., 5.],
         [3., 9.]]])



self.kernel_size 2
window_sliced tensor([[[1., 3.],
         [3., 8.]],

        [[3., 4.],
         [7., 4.]],

        [[3., 7.],
         [1., 0.]]])



self.kernel_size 2
window_sliced tensor([[[3., 7.],
         [8., 7.]],

        [[4., 2.],
         [4., 1.]],

        [[7., 5.],
         [0., 1.]]])



self.kernel_size 2
window_sliced tensor([[[7., 1.],
         

In [ ]:
img

In [ ]:
img[:,0:3,0:3]

In [ ]:
a = torch.randint(10, ( 3, 7, 10)).to(torch.float32)
input_width=a.shape[-1]
input_height=a.shape[-2]
kernel_size =5
stride = 1
padding=0


def image_shape(input_height, input_width, kernel_size, stride, padding):
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    return output_height, output_width

output_height,output_width=image_shape(input_height, input_width,kernel_size,stride, padding)

print(output_height,output_width)

In [ ]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

out_channels = kernel.shape[0]
in_channels  = kernel.shape[1]
kernel_size  = kernel.shape[2]
stride=2

print(f" \
out_channels {out_channels} \n \
in_channels  {in_channels}  \n \
kernel_size  {kernel_size}  \n \
"
)

In [ ]:
my_conv2d = Conv2dLoop(in_channels, out_channels, kernel_size, stride)
my_conv2d.set_kernel(kernel)

In [ ]:
input_tensor.shape[0]

In [ ]:
len(input_tensor.shape)

## Torch conv2d

In [ ]:
# in_channels = 3
# out_channels = 4
# kernel_size = 3
# stride = 1

in_channels = 3
out_channels = 4
kernel_size = 2
stride = 1

conv2d_torch = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

conv2d_torch(img)

In [ ]:
a = torch.randint(10, (2, 3, 7, 7)).to(torch.float32)
a.shape[0]
# for i in a:
#     print(i.shape)

In [ ]:
conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

In [ ]:
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

In [ ]:
torch_conv2d = Conv2d(3,4,5,1)
torch_conv2d(a).shape
torch_conv2d(a).shape

In [ ]:
a[0].shape

In [ ]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

calc_out_shape(a.shape, 4, 5, 1, 0)

In [ ]:
## Test input 

In [ ]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

in_channels = kernel.shape[1]
in_channels

In [ ]:
batch_size=2
input_height=4
input_width=4
stride=2


print(batch_size * in_channels * \
                            input_height * input_width)

input_tensor = torch.arange(0, batch_size * in_channels *
                            input_height * input_width,
                            out=torch.FloatTensor()) \
    .reshape(batch_size, in_channels, input_height, input_width)
print(input_tensor.shape)
input_tensor.shape[-3]